# Homework 2
Sam Friedman

## Problem 1
Find d-separated nodes. Assume that we have a Bayesian network graph $\mathit{G}$. Given a set of observed nodes $\mathbf{E}$, we want to find the set of nodes $\mathit{Y}$ that contains all nodes that are d-separated from the source node $X$.
    
    

In [32]:
# Create a Graph object
class Graph(object):
    ''' Graph object that has sets of nodes and edges '''
    def __init__(self, edges=set(), nodes=set(), observations=set()):
#         self.nodes = nodes
        self.parents = {}
        self.children = {}
        self.edges = edges
        self.obs = observations

        self.build_parent_and_child_sets()
        
        def build_parent_and_child_sets(self):
            for edge in self.edges:
                u, v = edge
                if v not in self.parents:
                    self.parents.update({v : set(u)})
                else:
                    self.parents[v].add(u)
                
                if u not in self.children:
                    self.children.update({u : set(v)})
                else:
                    self.children[u].add(v)
        
                
        #         if not edges:
#             self.edges = self.build_edge_set()
#         else:
#             self.edges
        
#     def build_edge_set(self):
#         nodes = self.nodes
#         edges = set()
#         for node in nodes:
#             for child in node.c:
#                 edges.add((node, child))
#             for parent in node.p:
#                 edges.add((parent, node))
                      
# Create Node object
# class Node(object):
#     ''' Node object with links to parent, children, and neighbor nodes '''
#     def __init__(self, parents=set(), children=set(), neighbors=set()):
#         self.p = parents
#         self.c = children
#         #self.n = neighbors
    
#     def add_parent(self, parent):
#         self.p.add(parent)
    
#     def add_child(self, child):
#         self.c.add(child)
    
    #def add_neighbor(self, neighbor):
    #    self.n.append(neighbor)
        
        
# Build Graph from homework 1, problem 3
nodes = set(('A','B','C','D','E','F','G','H','I','J','K','L','M'))
edges = set([('A','D'),('B','D'),('D','G'),('D','H'),('G','K'),('H','K'),\
             ('H','E'),('C','E'),('E','I'),('F','I'),('F','J'),('I','L'),\
             ('J','M')])
print(edges)


def d_separated():
    print('hello!')

d_separated()

{('F', 'I'), ('A', 'D'), ('D', 'H'), ('F', 'J'), ('G', 'K'), ('D', 'G'), ('B', 'D'), ('C', 'E'), ('J', 'M'), ('H', 'E'), ('H', 'K'), ('I', 'L'), ('E', 'I')}
hello!


In [29]:
nodes = set(('A','B','C','D','E','F','G','H','I','J','K','L','M'))
print(nodes)

{'L', 'D', 'I', 'E', 'M', 'K', 'A', 'G', 'C', 'B', 'F', 'H', 'J'}


In [22]:
s = set()
s.add('a')
print(s)
s.add('b')
print(s)
s.add('a')
s

l = []
l.append('b')
print(l)
l2 = set(l)
print(l2)
print(isinstance(l2, list))
isinstance(l2, set)

{'a'}
{'b', 'a'}
['b']
{'b'}
False


True